In [1]:
import boto3
from boto3.dynamodb.conditions import Key, Attr
import pandas as pd
import gspread
import json
import numpy as np
from decimal import Decimal
import datetime

In [2]:
def read_json(file='./iamcred.json'):
    response = dict()
    response['status'] = True
    try:
        with open(file, 'r') as file:
            response['data'] = json.load(file)
    except Exception as e:
        response['status'] = False
        response['data'] = e
    return response

In [3]:
def establish_connection(cred_file='./cred.json'):
    response = dict()
    response['status'] = True
    try:
        response['data'] = gspread.service_account(cred_file)
    except Exception as e:
        response['status'] = False
        response['data'] = e
    return response

In [4]:
def get_sheet(gc: gspread.service_account, sheet, worksheet=None):
    response = dict()
    response['status'] = True
    try:
        sheet = gc.open(sheet)
        if worksheet == None:
            worksheets = sheet.worksheets()
            if len(worksheets) >= 1:
                worksheet = worksheets[0].title
        response['data'] = sheet.worksheet(worksheet)
    except Exception as e:
        response['status'] = False
        response['data'] = e
    return response

In [5]:
def get_data_as_df(worksheet):
    response = dict()
    response['status'] = True
    try:
        df = pd.DataFrame(worksheet.get_all_records())
        df = df.set_index('turned_on')
        df = df.replace(r'^\s*$', np.nan, regex=True)
        df.fillna("None", inplace=True)
        response['data'] = df
    except Exception as e:
        response['status'] = False
        response['data'] = e
    return response

In [6]:
def get_dynamodb_resource(data):
    response = dict()
    response['status'] = True
    try:
        response['data'] = boto3.resource(
            service_name = data['service_name'],
            region_name = data['region_name'],
            aws_access_key_id = data['aws_access_key_id'],
            aws_secret_access_key = data['aws_secret_access_key']
        )
    except Exception as e:
        response['status'] = False
        response['data'] = e
    return response

In [7]:
def get_table(dynamodb, TableName):
    response = dict()
    response['status'] = True
    try:
        table = dynamodb.Table(TableName)
        test = table.creation_date_time
        response['data'] = table
    except Exception as e:
        response['status'] = False
        response['data'] = e
    return response

In [8]:
def create_data_packet(df, i):
    response = dict()
    response['status'] = True
    try:
        obj = df.iloc[i].to_dict()
        
        dt_string = df.index[i]
        format = "%Y-%m-%d %H:%M:%S"
        dt_object = datetime.datetime.strptime(dt_string, format)
        
        obj['turned_on'] = round(dt_object.timestamp())
        
        obj['voltage'] = Decimal(str(obj['voltage']))
        obj['current'] = Decimal(str(obj['current']))
        obj['power'] = Decimal(str(obj['power']))
        obj['span'] = int(obj['span'])
        
        response['data'] = obj
    except Exception as e:
        response['status'] = False
        response['data'] = e
    return response

In [9]:
def put_item(table, data):
    response = dict()
    response['status'] = True
    try:
        response['data'] = table.put_item(Item=data)
    except Exception as e:
        response['status'] = False
        response['data'] = e
    return response

In [10]:
def upload_data(table, df):
    response = dict()
    response['status'] = True
    try:
        for i in range(len(df)):
            resp = create_data_packet(df, i)
            if resp['status'] == False:
                raise Exception(resp['data'])
            data = resp['data']
            
            resp = put_item(table, data)
            if resp['status'] == False:
                raise Exception(resp['data'])
            
            print(i, 'uploaded!')
        response['data']='Successfull!'
    except Exception as e:
        response['status'] = False
        response['data'] = e
    return response

In [11]:
def delete_item(table, Key):
    response = dict()
    response['status'] = True
    try:
        response['data']=table.delete_item(Key=Key)
    except Exception as e:
        response['status'] = False
        response['data'] = e
    return response

In [12]:
def main(cred_file, iamcred, TableName, sheet, worksheet=None):
    response = dict()
    response['status'] = True
    try:
        resp = establish_connection(cred_file)
        if resp['status'] == False:
            raise Exception(resp['data'])
        gc = resp['data']
        
        resp = get_sheet(gc, sheet, worksheet)
        if resp['status'] == False:
            raise Exception(resp['data'])
        worksheet = resp['data']
        
        resp = get_data_as_df(worksheet)
        if resp['status'] == False:
            raise Exception(resp['data'])
        df = resp['data']
        
        resp = read_json(iamcred)
        if resp['status'] == False:
            raise Exception(resp['data'])
        creds = resp['data']
        
        resp = get_dynamodb_resource(creds)
        if resp['status'] == False:
            raise Exception(resp['data'])
        dynamodb = resp['data']
        
        resp = get_table(dynamodb, TableName)
        if resp['status'] == False:
            raise Exception(resp['data'])
        table = resp['data']
        
        resp = upload_data(table, df)
        if resp['status'] == False:
            raise Exception(resp['data'])

        response['data'] = resp['data']
        
    except Exception as e:
        response['status'] = False
        response['data'] = e
    return response

In [13]:
cred_file = './cred.json'
sheet = 'electricity'
worksheet = 'data'

iamcred = './iamcred.json'
TableName = 'Electricity'

main(cred_file, iamcred, TableName, sheet, worksheet)

0 uploaded!
1 uploaded!
2 uploaded!
3 uploaded!
4 uploaded!
5 uploaded!
6 uploaded!
7 uploaded!
8 uploaded!
9 uploaded!
10 uploaded!
11 uploaded!
12 uploaded!
13 uploaded!
14 uploaded!
15 uploaded!
16 uploaded!
17 uploaded!
18 uploaded!
19 uploaded!
20 uploaded!
21 uploaded!
22 uploaded!
23 uploaded!
24 uploaded!
25 uploaded!
26 uploaded!
27 uploaded!
28 uploaded!
29 uploaded!
30 uploaded!
31 uploaded!
32 uploaded!
33 uploaded!
34 uploaded!
35 uploaded!
36 uploaded!
37 uploaded!
38 uploaded!
39 uploaded!
40 uploaded!
41 uploaded!
42 uploaded!
43 uploaded!
44 uploaded!
45 uploaded!
46 uploaded!
47 uploaded!
48 uploaded!
49 uploaded!
50 uploaded!
51 uploaded!
52 uploaded!
53 uploaded!
54 uploaded!
55 uploaded!
56 uploaded!
57 uploaded!
58 uploaded!
59 uploaded!
60 uploaded!
61 uploaded!
62 uploaded!
63 uploaded!
64 uploaded!
65 uploaded!
66 uploaded!
67 uploaded!
68 uploaded!
69 uploaded!
70 uploaded!
71 uploaded!
72 uploaded!
73 uploaded!
74 uploaded!
75 uploaded!
76 uploaded!
77 upload

595 uploaded!
596 uploaded!
597 uploaded!
598 uploaded!
599 uploaded!
600 uploaded!
601 uploaded!
602 uploaded!
603 uploaded!
604 uploaded!
605 uploaded!
606 uploaded!
607 uploaded!
608 uploaded!
609 uploaded!
610 uploaded!
611 uploaded!
612 uploaded!
613 uploaded!
614 uploaded!
615 uploaded!
616 uploaded!
617 uploaded!
618 uploaded!
619 uploaded!
620 uploaded!
621 uploaded!
622 uploaded!
623 uploaded!
624 uploaded!
625 uploaded!
626 uploaded!
627 uploaded!
628 uploaded!
629 uploaded!
630 uploaded!
631 uploaded!
632 uploaded!
633 uploaded!
634 uploaded!
635 uploaded!
636 uploaded!
637 uploaded!
638 uploaded!
639 uploaded!
640 uploaded!
641 uploaded!
642 uploaded!
643 uploaded!
644 uploaded!
645 uploaded!
646 uploaded!
647 uploaded!
648 uploaded!
649 uploaded!
650 uploaded!
651 uploaded!
652 uploaded!
653 uploaded!
654 uploaded!
655 uploaded!
656 uploaded!
657 uploaded!
658 uploaded!
659 uploaded!
660 uploaded!
661 uploaded!
662 uploaded!
663 uploaded!
664 uploaded!
665 uploaded!
666 up

{'status': True, 'data': 'Successfull!'}